In [3]:
#Exercise 01 - Get monthly crime count by type
#data set location " https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2 "
!ls /home/veepee555/Downloads/
from pyspark.sql import SparkSession
from pyspark import *
spark = SparkSession \
.builder \
.appName('exerciseOne4Spark') \
.getOrCreate()



 big.csv		       'Praveen Vasudev_07_07_2019.docx'
 code-master		       'Praveen Vasudev_2017.docx'
 Codility-Python-master        'Praveen Vasudev_2019.docx'
 credentials.csv	       'Praveen Vasudev_2019.pdf'
 Crimes_-_2001_to_present.csv   sarakey.pem
 data-master		        Spark-practice-master
 iris.csv


In [23]:
crimesDF = spark \
.read \
.format("csv") \
.option("header","true") \
.option("mode","DROPMALFORMED") \
.option("inferSchema","true") \
.load('/home/veepee555/Downloads/Crimes_-_2001_to_present.csv')


In [7]:
type(crimesDF)

pyspark.sql.dataframe.DataFrame

In [26]:
crimesDF.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Historical Wards 2003-2015: integer (nullable = true)
 |-- Zip Codes: integer (nullable = tr

In [11]:
help(crimesDF.createOrReplaceTempView)

Help on method createOrReplaceTempView in module pyspark.sql.dataframe:

createOrReplaceTempView(name) method of pyspark.sql.dataframe.DataFrame instance
    Creates or replaces a local temporary view with this DataFrame.
    
    The lifetime of this temporary table is tied to the :class:`SparkSession`
    that was used to create this :class:`DataFrame`.
    
    >>> df.createOrReplaceTempView("people")
    >>> df2 = df.filter(df.age > 3)
    >>> df2.createOrReplaceTempView("people")
    >>> df3 = spark.sql("select * from people")
    >>> sorted(df3.collect()) == sorted(df2.collect())
    True
    >>> spark.catalog.dropTempView("people")
    
    .. versionadded:: 2.0



In [20]:
crimesDF.createOrReplaceTempView('ChicagoCrimes')
spark.sql("describe ChicagoCrimes").show()
spark.sql("show tables").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                  ID|      int|   null|
|         Case Number|   string|   null|
|                Date|   string|   null|
|               Block|   string|   null|
|                IUCR|   string|   null|
|        Primary Type|   string|   null|
|         Description|   string|   null|
|Location Description|   string|   null|
|              Arrest|  boolean|   null|
|            Domestic|  boolean|   null|
|                Beat|      int|   null|
|            District|      int|   null|
|                Ward|      int|   null|
|      Community Area|      int|   null|
|            FBI Code|   string|   null|
|        X Coordinate|      int|   null|
|        Y Coordinate|      int|   null|
|                Year|      int|   null|
|          Updated On|   string|   null|
|            Latitude|   double|   null|
+--------------------+---------+-------+
only showing top

In [34]:
#Get monthly count of primary crime type, sorted by month in ascending and number of crimes per type in descending order

spark.sql("select  \
concat(substr(Date,7,4),substr(Date,7,4)), \
'Primary Type', count(1)   \
from ChicagoCrimes  \
group by concat(substr(Date,7,4),substr(Date,7,4)) ,'Primary Type' \
order by concat(substr(Date,7,4),substr(Date,7,4)) asc,'Primary Type' desc , count(1) desc  ").show()

+----------------------------------------------------+------------+--------+
|concat(substring(Date, 7, 4), substring(Date, 7, 4))|Primary Type|count(1)|
+----------------------------------------------------+------------+--------+
|                                            20012001|Primary Type|  485755|
|                                            20022002|Primary Type|  486757|
|                                            20032003|Primary Type|  475947|
|                                            20042004|Primary Type|  469385|
|                                            20052005|Primary Type|  453719|
|                                            20062006|Primary Type|  448116|
|                                            20072007|Primary Type|  437019|
|                                            20082008|Primary Type|  427068|
|                                            20092009|Primary Type|  392702|
|                                            20102010|Primary Type|  370331|

In [35]:
resultDF=spark.sql("select  \
concat(substr(Date,7,4),substr(Date,7,4)), \
'Primary Type', count(1)   \
from ChicagoCrimes  \
group by concat(substr(Date,7,4),substr(Date,7,4)) ,'Primary Type' \
order by concat(substr(Date,7,4),substr(Date,7,4)) asc,'Primary Type' desc , count(1) desc  ")

In [37]:
help(resultDF.write)

Help on DataFrameWriter in module pyspark.sql.readwriter object:

class DataFrameWriter(OptionUtils)
 |  DataFrameWriter(df)
 |  
 |  Interface used to write a :class:`DataFrame` to external storage systems
 |  (e.g. file systems, key-value stores, etc). Use :func:`DataFrame.write`
 |  to access this.
 |  
 |  .. versionadded:: 1.4
 |  
 |  Method resolution order:
 |      DataFrameWriter
 |      OptionUtils
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, df)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  bucketBy(self, numBuckets, col, *cols)
 |      Buckets the output by the given columns.If specified,
 |      the output is laid out on the file system similar to Hive's bucketing scheme.
 |      
 |      :param numBuckets: the number of buckets to save
 |      :param col: a name of a column, or a list of names.
 |      :param cols: additional names (optional). If `col` is a list it should be empty.
 |      
 |      .. not

In [41]:
resultDF \
.write \
.format("csv") \
.option("compression", "org.apache.hadoop.io.compress.GzipCodec") \
.option("sep", "\t") \
.save("/home/veepee555/Downloads/result.tsv")


